In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
print(df.shape)

(20800, 5)


In [ ]:
###Drop Nan Values
df=df.dropna()

print(df.shape)

(18285, 5)


In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
### Vocabulary size
vocabulary_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
print(messages['title'][0])

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
onehot_repr=[one_hot(words,vocabulary_size)for words in corpus] 

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocabulary_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 18s 71ms/step - loss: 0.4815 - accuracy: 0.7531 - val_loss: 0.2010 - val_accuracy: 0.9198
Epoch 2/10
192/192 [==============================] - 14s 71ms/step - loss: 0.1442 - accuracy: 0.9469 - val_loss: 0.1863 - val_accuracy: 0.9246
Epoch 3/10
192/192 [==============================] - 13s 70ms/step - loss: 0.0979 - accuracy: 0.9652 - val_loss: 0.2008 - val_accuracy: 0.9205
Epoch 4/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0737 - accuracy: 0.9756 - val_loss: 0.2295 - val_accuracy: 0.9170
Epoch 5/10
192/192 [==============================] - 13s 67ms/step - loss: 0.0560 - accuracy: 0.9823 - val_loss: 0.2962 - val_accuracy: 0.9155
Epoch 6/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0491 - accuracy: 0.9853 - val_loss: 0.2874 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0307 - accuracy: 0.9896 - val_loss: 0.3531 - val_accuracy:

In [ ]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3088,  331],
       [ 191, 2425]])

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy Score is : ",accuracy_score(y_test,y_pred))

Accuracy Score is :  0.9135045567522784
